In [ ]:
import numpy as np
import pandas as pd
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [ ]:
trainDF = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', 
                 usecols=['Target', 'Asset_ID','timestamp'], dtype={'Asset_ID': 'int8'})
trainDF['datetime'] = pd.to_datetime(trainDF['timestamp'], unit='s')
trainDF = trainDF.set_index('datetime').drop('timestamp', axis=1)
trainDF = trainDF[(trainDF.index.year == 2021) & (trainDF.index.month > 5)]
dfs = {asset_id: trainDF[trainDF['Asset_ID'] == asset_id].resample('1min').interpolate().copy() for asset_id in trainDF['Asset_ID'].unique()}
del trainDF
for df_test, df_pred in iter_test:
    df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
    for _, row in df_test.iterrows():
        try:
            df = dfs[row['Asset_ID']]
            closest_train_sample = df.iloc[df.index.get_loc(row['datetime'], method='nearest')]
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = closest_train_sample['Target']
        except:
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
    df_pred['Target'] = df_pred['Target'].fillna(0)
    env.predict(df_pred)